In [4]:
import requests as req
import re
import datetime
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pymongo
from selenium import webdriver
import time  

In [5]:
# 몽고DB 서버 연결
username = 'ubuntu'
password = 'ubuntu'
conn = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))

In [6]:
# db, collection 연결(없으면 생성)
# python과 연동되었을 때, db와 collection의 경우, document insert 싯점에 생성됨
mongodb = conn.cine21hw
#collection 객체 변수 = mongoDB.collection name(없으면 만들어짐)
actor_collection = mongodb.actor_collection

In [7]:
# actor 정보가 있는지 확인하기
actor_list = actor_collection.find()
for actor in actor_list:
    print(actor['actor'])

조우진
김윤석
조인성
성동일
이제훈
허준호
김소진
조우진
김윤석
조인성
성동일
이제훈
허준호
김소진


In [8]:
# request url
cine21_url = 'http://www.cine21.com/rank/person/content'
year = "2020-10" 
# year = "2020-11" 1년치의 정보를 가져온다. 

# post 방식으로 데이터를 가져오기 위한 request 시 필요 정보
# 브라우저의 개발자 모드의 network tab의 form 정보 
conditions = dict()
conditions['section'] = 'actor'
conditions['period_start'] = year
conditions['gender'] = 'all'
conditions['page'] = 1

# post 방식으로 request 요청
response = req.post(cine21_url, data = conditions)

In [ ]:
print(response)
print(response.content)

In [9]:
soup = bs(response.content, "html.parser") 

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time  

#스크롤
options = webdriver.ChromeOptions()
options.add_argument('lang=ko_KR')
options.add_argument("no-sandbox") 
#options.add_argument('headless')

driver_path = '/home/ubuntu/datagather/selenium/chromedriver'
chrome = webdriver.Chrome(driver_path, options=options)

url =  "http://www.cine21.com/rank/person"
chrome.get(url)

def page_loag():

    interval = 2

    chrome.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 현재 document의 높이 반환
    prev_height = chrome.execute_script("return document.body.scrollHeight")

    while True: # break를 항상 줘야 한다. 

        # 스크롤을 현재 페이지 가장 아래로 내림
        chrome.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        # 페이지 로딩 대기
        time.sleep(interval)
        # 현재 문서 높이를 가져와서 저장
        curr_height = chrome.execute_script("return document.body.scrollHeight")
        #print(prev_height,curr_height)
        if curr_height == prev_height:
            break
        # 이전 높이 변경
        prev_height = curr_height

    print("스크롤 완료")
    
def next_page():
    items = chrome.find_elements_by_css_selector("li[class ^= people_li")
    time.sleep(1)
    next_page = chrome.find_element_by_css_selector("a[class ^= btn_next]")
    next_page.click()

    time.sleep(1)

/tmp/ipykernel_3864/291280763.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = webdriver.Chrome(driver_path, options=options)


In [16]:
# 배우, 흥행지수, 영화 제목 가져오기 
li_alls = soup.select("li.people_li")
#print(li_alls)
actors_list= []
actors_rate= []
movie_list= []
for li_all in li_alls:
    actor_li = li_all.select_one("div.name").get_text(strip=True)
    actors_list.append(re.sub("\(\w+\)", "", actor_li))
#print(actors_list)  
    actor_ra = li_all.select_one("strong")
    actors_rate.append(re.sub("<.*?>", "", str(actor_ra)))
#print(actors_rate)
    movie_li = li_all.select("ul.mov_list span")
    movie_list.append(re.sub("<.*?>", "", str(movie_li)))
#print(movie_list)
    #movie_list.append(movie_li)
#print(movie_list)


['[발신제한, 도굴, 서복, 국가부도의 날, 봉오동 전투, 강철비]', '[모가디슈, 당신, 거기 있어줄래요, 화이 : 괴물을 삼킨 아이, 1987, 도둑들]', '[모가디슈, 클래식]', '[담보, 수상한 그녀]', '[도굴, 사냥의 시간, 건축학개론, 아이 캔 스피크, 파수꾼, 박열]', '[모가디슈, 천문: 하늘에 묻는다, 국가부도의 날]', '[모가디슈, 스프링 송, 아이 캔 스피크, 남산의 부장들]']


In [17]:
# 배우 상세정보 가져오기 
actors = soup.select("li.people_li div.name")
actors_detail_info = list()


for actor in actors:
    url_link = "http://www.cine21.com" + actor.select_one('a').attrs['href']
    res = req.get(url_link)
    # soup_actor: 배우 상세정보를 저장하는 soup 객체(html 문서)
    soup_actor = bs(res.content, "html.parser")
    # 상세정보 값
    default_info = soup_actor.select_one("ul.default_info")
    #print(default_info )

    # 배우 상세정보를 list로  return받음
    actor_info_dict = dict()
    actor_details = default_info.select("li")
    for actor_detail in actor_details:
        # span문 안에 text 뽑기
        actor_detail_key = actor_detail.select_one("span.tit").get_text(strip=True)
        actor_detail_value = re.sub("<span.*?>.*?</span>", "", str(actor_detail))
        actor_detail_value = re.sub("<.*?>", "", actor_detail_value).strip()
        actor_info_dict[actor_detail_key] = actor_detail_value
    actors_detail_info.append(actor_info_dict)
# 배우들의 상세정보 출력 
print(actors_detail_info)

[{'다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}, {'직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}, {'다른 이름': '趙寅成', '직업': '배우', '생년월일': '1981-07-28', '성별': '남', '신장/체중': '186cm, 72kg', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)', '취미': '모자모으기, 영화보기, 농구', '특기': '농구, 태권도(공인 4단)', '소속사': '싸이더스 HQ'}, {'직업': '배우', '생년월일': '1967-04-27', '성별': '남', '신장/체중': '177cm, 78kg', '학교': '유한대학 기계설계학'}, {'직업': '배우', '생년월일': '1984-07-04', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.leejehoon', '신장/체중': '176cm', '학교': '한국예술종합학교 연극원 연기과', '특기': '댄스, 노래, 무술', '소속사': '㈜사람엔터테인먼트'}, {'원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}, {'직업': '배우', '성별': '여'}]


In [18]:
for num,actor in enumerate(actors_list):
    actor_collection.insert_one(
        {
            "actor": actor,
            "actor_detail": actors_detail_info[num],
            "actor_rate": actors_rate[num],
            "date": year,
            "movies_list": movie_list[num]
        }
    )

In [20]:
actors_info = list()

for num, actor in enumerate(actors_list):
    actors_info.append(
        {
            "actor": actor,
            "actor_detail": actors_detail_info[num],
            "actor_rate": actors_rate[num],
            "date": year,
            "movies_list": movie_list[num]
        }
    )
print(actors_info)
actor_collection.insert_many(actors_info)

[{'actor': '조우진', 'actor_detail': {'다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}, 'actor_rate': '41,827', 'date': '2020-10', 'movies_list': '[발신제한, 도굴, 서복, 국가부도의 날, 봉오동 전투, 강철비]'}, {'actor': '김윤석', 'actor_detail': {'직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}, 'actor_rate': '37,291', 'date': '2020-10', 'movies_list': '[모가디슈, 당신, 거기 있어줄래요, 화이 : 괴물을 삼킨 아이, 1987, 도둑들]'}, {'actor': '조인성', 'actor_detail': {'다른 이름': '趙寅成', '직업': '배우', '생년월일': '1981-07-28', '성별': '남', '신장/체중': '186cm, 72kg', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)', '취미': '모자모으기, 영화보기, 농구', '특기': '농구, 태권도(공인 4단)', '소속사': '싸이더스 HQ'}, 'actor_rate': '32,659', 'date': '2020-10', 'movies_list': '[모가디슈, 클래식]'}, {'actor': '성동일', 'actor_detail': {'직업': '배우', '생년월일': '1967-04-27', '성별': '남', '신장/체중': '177cm, 78kg', '학교': '유한대학 기계설계학'}, 'actor_rate': '30,628', 'date': '2020-10', 'movies_list': '[담보, 수상한 그녀]'}, {'actor': '이제훈', 'actor_detail': {'직업': 